In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.simplefilter("ignore")

df = pd.read_excel("DATA COMPENSATION.xlsx")


#   COMPENSATION ENGINE 


class CompensationEngine:
    def __init__(self):
        
        # PRODUCT MAPPING
        self.accounts = {
            "Business": {
                1096: "Business Pay As You Go",
                1098: "Business Platinum",
                1095: "Business Current Account",
                1097: "Business Gold",
                1201: "Current Corporate",
                1206: "Non-profit Institution",
                1207: "Education",
                1208: "FBO",
                1211: "Institutional",
            },
            "Personal current": {
                1087: "Gold Pay As You Go",
                1085: "Student Account",
                1094: "Homeward Account",
                1039: "Gold All in one",
                1081: "Work Pay as you Go",
                1082: "Salary Account",
                1086: "Diplomatic Current",
                1137: "Go Banking",
            },
            "Personal savings": {
                6011: "Business Saver",
                6055: "Premier Savings",
                6056: "Junior Savings",
                6058: "Gold Savings",
                1084: "Wiser Account"
            },
            "Affluent": {
                1083: "Platinum Pay as you Go",
                1001: "Platinum All in one",
                1093: "Platinum Flexi"
            }
        }

        self.loans = {
            "Personal Loans": {
                3103: "LOAN-PERSONAL",
                3708: "Personal Unsecured Non Scheme",
                3709: "Personal Unsecured Scheme",
                3116: "Salary Advance"
            }
        }

        self.mortgages = {
            "Mortgage": {
                3713: "Cash Covered",
                3221: "Market Housing-AHP",
                3704: "Equity Release",
                3698: "Affordable Housing"
            }
        }

        self.asset = {
             "Asset Finance":  {
                3724: "Saloon",
                3728: "Trailers",
                3715: "Commercial vehicles",
                3716: "Contractor Equipment",
                3727: "Tractors",
                3720: "Psv Buses",
                3725: "School Buses",
                3100: "Loan-Asset Finance",
                3721: "Psv Matatu"
             }
        }

        # FIXED PAY
        self.starting_retainer = 10000
        self.annual_increment = 5000
        self.max_retainer = 35000
        self.airtime_allowance = 4500
        self.training_allowance = 5000

        # ACCOUNT RULES
        self.account_min_threshold = {
            "Business": 20000,
            "Personal current": 10000,
            "Personal savings": 10000,
            "Affluent": 10000
        }

        self.account_absolute = {
            "Business": {"bundled": 2500, "unbundled": 1500},
            "Personal current": {"bundled": 2000, "unbundled": 1000},
            "Personal savings": 500,
            "Affluent": 2000
        }

        # ASSET RATES
        self.asset_commission_rates = {
            "used_bundled_gt_10m": 0.015,
            "used_bundled_lt_10m": 0.01,
            "used_unbundled": 0.0075,
            "new_heavy": 0.005,
        }

        # MORTGAGE
        self.mortgage_commission = {
            "approval": 0.005,
            "disbursement": 0.005
        }

        # LOANS
        self.personal_loans_commission = {
            "new": 0.01,
            "existing": 0.005
        }

  
    # FIXED PAY
    
    def compute_fixed_pay(self, date_joined):

        if isinstance(date_joined, str):
            date_joined = pd.to_datetime(date_joined)

        today = datetime.today()

        months = (today.year - date_joined.year) * 12 + (today.month - date_joined.month)
        years = max((months-12) // 12, 0)

        retainer_component = min(
            self.starting_retainer + years * self.annual_increment,
            self.max_retainer
        )
        training_component = self.training_allowance if months < 3 else 0
        airtime_component = self.airtime_allowance 

        return retainer_component + self.airtime_allowance + self.training_allowance

   
    # ACCOUNT REMUNERATION
 
    def account_remuneration(self, row):

        product_code = row.get("PRODUCT CODE")
        deposits = row.get("TOTAL CURRENT DEPOSIT", 0)
        bundled_status = str(row.get("BUNDLED CHECK", "")).lower()

        # Identify account type
        account_type = None
        for acc_type, mapping in self.accounts.items():
            if product_code in mapping:
                account_type = acc_type
                break

        if account_type is None:
            return 0

        # Threshold check
        if deposits < self.account_min_threshold[account_type]:
            return 0

        if bundled_status not in ["bundled", "unbundled"]:
            bundled_status = "unbundled"

        if account_type == "Affluent":
            return 2000

        if account_type in ["Business", "Personal current"]:
            return self.account_absolute[account_type][bundled_status]

        return self.account_absolute["Personal savings"]

   
    # ASSET COMMISSION
    
    def asset_commission(self, row):

        product_code = row.get("PRODUCT CODE")
        amount = row.get("NET", 0)
        condition = str(row.get("CONDITION", "")).lower()
        bundled_status = str(row.get("BUNDLED CHECK", "")).lower()

        if pd.isna(product_code) or amount <= 0:
            return 0

        # Only asset-finance products
        asset_codes = self.asset["Asset Finance"].keys()
        if product_code not in asset_codes:
            return 0

        if bundled_status not in ["bundled", "unbundled"]:
            bundled_status = "unbundled"

        #  commission key
        if condition == "used":
            if bundled_status == "bundled":
                key = "used_bundled_gt_10m" if amount > 10_000_000 else "used_bundled_lt_10m"
            else:
                key = "used_unbundled"

        elif condition in ["new", "heavy", "equipment"]:
            key = "new_heavy"

        else:
            return 0

        return amount * self.asset_commission_rates[key]

  
    # MORTGAGE
   
    def mortgage_compensation(self, row):

        product_code = row.get("PRODUCT CODE")
        amount = row.get("NET", 0)
        description = str(row.get("CONDITION", "")).lower()

        mortgage_codes = self.mortgages["Mortgage"].keys()

        if product_code not in mortgage_codes or amount <= 0:
            return 0

        if "approval" in description:
            rate = self.mortgage_commission["approval"]
        else:
            rate = self.mortgage_commission["disbursement"]

        return amount * rate

    
    # PERSONAL LOANS
    
    def personal_loan_compensation(self, row):

        product_code = row.get("PRODUCT CODE")
        amount = row.get("NET", 0)
        outstanding = row.get("OUTSTANDING", 0)

        loan_codes = self.loans["Personal Loans"].keys()

        if product_code not in loan_codes or amount <= 0:
            return 0

        if outstanding == 0:
            rate = self.personal_loans_commission["new"]
        else:
            rate = self.personal_loans_commission["existing"]

        return amount * rate



#  ENGINE


engine = CompensationEngine()

def compute_compensation_breakdown(row):
    return pd.Series({
        "accounts_comp_calc": engine.account_remuneration(row),
        "asset_comp_calc": engine.asset_commission(row),
        "mortgage_comp_calc": engine.mortgage_compensation(row),
        "loan_comp_calc": engine.personal_loan_compensation(row)
    })

comp_df = df.join(df.apply(compute_compensation_breakdown, axis=1))

# FIXED PAY
fixed_pay_table = (
    df.groupby("DSA")["DATE JOINED"]
      .first()
      .apply(engine.compute_fixed_pay)
      .reset_index()
      .rename(columns={"DATE JOINED": "fixed_pay_calc"})
)

# STAFF SUMMARY
staff_month_summary = (
    comp_df.groupby("DSA")[["accounts_comp_calc","asset_comp_calc",
                            "mortgage_comp_calc","loan_comp_calc"]]
    .sum()
    .reset_index()
    .merge(fixed_pay_table, on="DSA")
)

staff_month_summary["total_compensation_calc"] = (
    staff_month_summary["fixed_pay_calc"]
    + staff_month_summary["accounts_comp_calc"]
    + staff_month_summary["asset_comp_calc"]
    + staff_month_summary["mortgage_comp_calc"]
    + staff_month_summary["loan_comp_calc"]
)



In [7]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"Compensation_Report__{timestamp}.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    comp_df.to_excel(writer, sheet_name="Detailed_Data", index=False)
    staff_month_summary.to_excel(writer, sheet_name="Staff_Month_Summary", index=False)

print("Excel report generated:", output_file)

Excel report generated: Compensation_Report__20251128_101121.xlsx
